## Interview "Takehome" Coding Test

### Problem(s)

1. find and download 1000 (or more) images of handwritten digits and correct labels for each (0 - 9)
2. create a dataframe containing the image pixels for each image in a single row (vector) of integers or floats
3. use SVD (PCA) algoirthms in Scikit-Learn (or any package) to reduce the dimensions of those vectors below 13-D
4. cluster those vectors with a raw python function or script (using any algorithm you are familiar with, or one of your own design). Do **NOT** use sklearn or any other package not included in python (besides numpy and pandas) for this.
5. use your cluster IDs to assign digit labels to each of the 1000 vectors (images). You **should** use the correct labels that you downloaded to make this assignment. But you should **NOT** use any machine learning model. Like #4 this should be a pure-python exercise. 




### Solution 1

In [14]:
!pip install requests
!pip uninstall -y mnist
!pip install python-mnist

Uninstalling mnist-0.2.1:
  Successfully uninstalled mnist-0.2.1


In [1]:
import os
os.path.basename
dirname = 'MNIST'
curdir = os.path.basename(os.path.abspath(os.path.curdir))
print(curdir)
print(os.path.abspath(os.path.curdir))

# conditionals required to make this idempotent (so it can be run many times)
if curdir != dirname:
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    os.chdir(dirname)
print(os.path.abspath(os.path.curdir))

13-Review
/home/hobs/src/civicu-machine-learning/lessons/13-Review
/home/hobs/src/civicu-machine-learning/lessons/13-Review/MNIST


In [8]:
import requests
import gzip

url = 'http://yann.lecun.com/exdb/mnist/'
filenames = ['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte']

for filename in filenames:
    resp = requests.get(url + filename + '.gz', stream=True)
    with open(filename + '.gz', 'wb') as fout:
        print(resp.url + ' -> ' + filename + '.gz')
        for chunk in resp.iter_content(chunk_size=1024):
            if chunk:
                fout.write(chunk)
    with gzip.open(filename + '.gz', 'rb') as fin:
        print(filename + '.gz -> ' + filename)
        with open(filename, 'wb') as fout:
            fout.write(fin.read())


http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz -> t10k-images-idx3-ubyte.gz
t10k-images-idx3-ubyte.gz -> t10k-images-idx3-ubyte
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz -> t10k-labels-idx1-ubyte.gz
t10k-labels-idx1-ubyte.gz -> t10k-labels-idx1-ubyte


In [9]:
!pwd
!ls -al

/home/hobs/src/civicu-machine-learning/lessons/13-Review/MNIST
total 10968
drwxr-xr-x 2 hobs hobs    4096 Oct  9 15:30 .
drwxr-xr-x 4 hobs hobs    4096 Oct  9 15:41 ..
-rw-r--r-- 1 hobs hobs 7840016 Oct  9 15:41 t10k-images-idx3-ubyte
-rw-r--r-- 1 hobs hobs 1648877 Oct  9 15:41 t10k-images-idx3-ubyte.gz
-rw-r--r-- 1 hobs hobs 1648877 Oct  9 15:30 t10k-images-idx3-ubyte.orig.gz
-rw-r--r-- 1 hobs hobs   10008 Oct  9 15:41 t10k-labels-idx1-ubyte
-rw-r--r-- 1 hobs hobs    4542 Oct  9 15:41 t10k-labels-idx1-ubyte.gz
-rw-r--r-- 1 hobs hobs    4542 Oct  9 15:30 t10k-labels-idx1-ubyte.orig.gz


In [24]:
from mnist import MNIST

db = MNIST('.')
# FileNotFoundError: [Errno 2] No such file or directory: './t10k-labels-idx1-ubyte'
images, labels = db.load_testing()
print(type(images))
print(len(images))
print(type(labels))
print(len(labels))

<class 'list'>
10000
<class 'array.array'>
10000


In [25]:
import pandas as pd

image_width = int(len(list(images[0])) ** .5)
images = pd.DataFrame(images, columns=[(int(i / image_width), i % image_width) for i in range(image_width * image_width)])
images.columns = pd.MultiIndex.from_tuples(images.columns.values)

In [26]:
images.head()

0                             ... 27                           
  0  1  2  3  4  5  6  7  8  9  ... 18 19 20 21 22 23 24 25 26 27
0  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
3  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
4  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0

[5 rows x 784 columns]

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components=12)
images_12d = pd.DataFrame(pca.fit_transform(images), index=images.index) 
images_12d.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-411.260988,-686.561634,-51.113577,-242.567475,-483.546401,-380.058217,425.765883,-585.976347,-220.642997,274.633910,-30.791558,123.300488
1,58.065957,983.170050,8.884185,397.278766,-171.260510,447.792964,127.850263,-198.173698,210.817219,142.548314,111.174932,145.282729
2,-935.105330,459.074619,319.994431,262.530081,-469.900986,-126.968181,-32.387033,153.782935,84.450495,153.541369,-45.159547,-65.882819
3,1255.666520,-106.992651,121.966650,-246.650263,-318.412646,405.657428,627.279052,-128.263211,23.016377,115.330363,-175.939612,-530.155983
4,132.887072,-744.542034,-61.202333,703.882739,-129.236995,-279.900741,-509.313974,-281.314253,-226.677997,-51.295014,-266.023374,174.301325


### [Standard K-Means Algorithm](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm)

1. chose K starting vectors (a random sample is fine)
2. go through the entire list of vectors and assign each one to the cluster whose mean is closest to the point
3. record the previous means for the clusters and calculate new means (centroids) for each cluster
4. if any of the new means are significantly different from the old means then go back to step 2 and do it all again

In [ ]:
K = 10
N = len(images_12d)
D = len(images_12d.columns)

previous_means = pd.np.zeros(K)
means = images_12d.sample(K)  # randomly chose K starting points for the means
clusters = [list() for i in range(K)]  # list of point IDs in each cluster
done = False

# if the means move more than a small portion of the average squared sum of the vectors then try to refine them some more
epsilon = 1e-11 * pd.np.sum(images_12d ** 2) / N / D 

while any(change > epsilon):
    for i, v in 
    closest = sorted([(pd.np.sqrt(pd.np.sum((v - mean) ** 2)), i) for i, v in images_12.iterrows()])[:]
    change = pd.np.sum([(previous_means[i] - means[i]) ** 2 / D  for i in range(K)])

In [32]:
for i in t.index.levels.:
    print(i)

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
           dtype='int64')
Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
           dtype='int64')
